In [1]:
from retraction_audit.audit import RetractionLookup
import os

os.environ["RETRACTION_WATCH_EMAIL"] = "james.sharpsteen@gmail.com"
os.environ["NCBI_AUTH_EMAIL"] = "james.sharpsteen@gmail.com"

lookup = RetractionLookup(True)

In [18]:
from wags_tails import CustomData
from wags_tails.utils.downloads import handle_zip, download_http
from wags_tails.utils.versioning import DATE_VERSION_PATTERN
import requests
import datetime
import json

def _latest_version_cb() -> str:
    url = f"https://api.github.com/repos/vanallenlab/moalmanac-db/releases"
    response = requests.get(url)
    response.raise_for_status()
    data = response.json()
    return datetime.datetime.strptime(data[0]["tag_name"], "v.%Y-%m-%d").replace(tzinfo=datetime.timezone.utc).strftime(DATE_VERSION_PATTERN)
    
def _download_cb(version, outfile_path) -> None:
    tqdm_params = {
        "disable": False,
        "unit": "B",
        "ncols": 80,
        "unit_divisor": 1024,
        "unit_scale": True,
    }
    formatted_version = datetime.datetime.strptime(version, DATE_VERSION_PATTERN).replace(tzinfo=datetime.timezone.utc).strftime("v.%Y-%m-%d")
    download_http(
        f"https://github.com/vanallenlab/moalmanac-db/archive/refs/tags/{formatted_version}.zip",
        outfile_path,
        tqdm_params=tqdm_params,
        handler=handle_zip
    )
    pass
    
data_fetcher = CustomData(
    "moalmanac",
    "json",
    _latest_version_cb,
    _download_cb,
)

moa = data_fetcher.get_latest()[0]
with open(moa) as f:
    data = json.load(f)

In [19]:
pmids = {d["pmid"] for d in data} - {""}

In [20]:
retracted_ev = []
for pmid in pmids:
    retraction_record = lookup.get_retraction_by_pmid(str(pmid))
    if retraction_record:
        retracted_ev.append(retraction_record)

In [21]:
print(len(retracted_ev))
from tools import store_results, get_latest_results
store_results("MOAlmanac", len(retracted_ev), retracted_ev, len(pmids))
get_latest_results()

0


[('CIViC', 1, 3667, '2024-03-15T00:03:17.308166+00:00'),
 ('DGIDB', 20, 11403, '2024-03-15T00:08:13.144921+00:00'),
 ('PMKB', 0, 1562, '2024-03-15T01:45:21.760435+00:00'),
 ('PharmGKB', 2, 17150, '2024-03-15T13:35:38.913493+00:00'),
 ('DOKB', 2, 4318, '2024-03-15T14:38:21.085659+00:00'),
 ('MOAlmanac', 0, 166, '2024-03-17T00:31:14.361317+00:00')]